# Librerias

In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Strategy
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
from datetime import datetime
import itertools
import MetaTrader5 as mt5
import pytz
from backtest.utils import plot_full_equity_curve, walk_forward
import numpy as np
import random
random.seed(42)

# Obtener datos

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
print(tickers)

['AAPLm', 'ABTm', 'AMZNm', 'ATVIm', 'BACm', 'BMYm', 'Cm', 'CMCSAm', 'CSCOm', 'CSXm', 'CVSm', 'EBAYm', 'GILDm', 'GOOGLm', 'HDm', 'IBMm', 'INTCm', 'ISRGm', 'JNJm', 'JPMm', 'KOm', 'LINm', 'LLYm', 'LMTm', 'MAm', 'MCDm', 'MDLZm', 'MMMm', 'MOm', 'MRKm', 'MSm', 'NFLXm', 'NKEm', 'ORCLm', 'PEPm', 'PFEm', 'PGm', 'PMm', 'PYPLm', 'REGNm', 'SBUXm', 'Tm', 'TMOm', 'TMUSm', 'TSLAm', 'UNHm', 'UPSm', 'Vm', 'VZm', 'WFCm', 'WMTm', 'XOMm', 'AMCm', 'BBm', 'BBBYm', 'BEKEm', 'BIDUm', 'BILIm', 'BRQSm', 'BYNDm', 'CANm', 'EDUm', 'FTNTm', 'FUTUm', 'IQm', 'JDm', 'LIm', 'NIOm', 'NTESm', 'PDDm', 'RLXm', 'TALm', 'TIGRm', 'TMEm', 'TSMm', 'VIPSm', 'XPEVm', 'YUMCm', 'ZTOm', 'METAm']


In [5]:
intervals = [
    mt5.TIMEFRAME_H4,
]

parameter_combinations = list(itertools.product(
    tickers, intervals
))

symbols = {}

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
utc_from = datetime(2010, 10, 1, tzinfo=timezone)
utc_to = datetime(2024, 9, 1, tzinfo=timezone)

for ticker, interval in parameter_combinations:
    print(ticker)
    # Obtener las tasas históricas
    rates = mt5.copy_rates_range(ticker, interval, utc_from, utc_to)
    
    # Crear DataFrame con las tasas
    df = pd.DataFrame(rates)
    
    # Convertir el tiempo de segundos a formato datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')


    # Renombrar columnas para el ticker principal
    df = df.rename(columns={
        'time': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'tick_volume': 'Volume'
    }).set_index('Date')


    df.index = df.index.tz_localize('UTC').tz_convert('UTC')
    
    utc_to_ts = pd.Timestamp(utc_to).tz_convert('UTC')

    days = (utc_to_ts - df.index[-1]).days
    if days >= 4:
        tickers.remove(ticker)
        print(f'se descarto {ticker}')
        continue

    if ticker not in symbols.keys():
        symbols[ticker] = {}
        symbols[ticker][interval] = {}

    symbols[ticker][interval] = df


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()



MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AAPLm
ABTm
AMZNm
ATVIm
se descarto ATVIm
BACm
BMYm
Cm
CMCSAm
CSCOm
CSXm
CVSm
EBAYm
GILDm
GOOGLm
HDm
IBMm
INTCm
ISRGm
JNJm
JPMm
KOm
LINm
LLYm
LMTm
MAm
MCDm
MDLZm
MMMm
MOm
MRKm
MSm
NFLXm
NKEm
ORCLm
PEPm
PFEm
PGm
PMm
PYPLm
REGNm
SBUXm
Tm
TMOm
TMUSm
TSLAm
UNHm
UPSm
Vm
VZm
WFCm
WMTm
XOMm
AMCm
se descarto AMCm
BBm
BBBYm
se descarto BBBYm
BEKEm
BIDUm
BILIm
BRQSm
se descarto BRQSm
BYNDm
se descarto BYNDm
CANm
se descarto CANm
EDUm
FTNTm
FUTUm
IQm
JDm
LIm
NIOm
NTESm
PDDm
RLXm
TALm
TIGRm
se descarto TIGRm
TMEm
TSMm
VIPSm
XPEVm
YUMCm
ZTOm
METAm


True

In [6]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates

{16388: Timestamp('2021-10-27 16:00:00+0000', tz='UTC')}

# WFO

In [7]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class DayPerWeek(Strategy):
    risk = 1
    day_to_buy = 3
    percentage_price_sl = 5
    sma_period = 200
    rsi_period = 2
    rsi_upper_threshold = 90
    pip_value = 0.1
    
    def init(self):
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.rsi = self.I(
            ta.RSI, self.data.Close, 2
        )
        
    def next(self):
        today = self.data.index[-1]
        if self.position:
            if self.position.is_long:
                if self.rsi > self.rsi_upper_threshold:
                    self.position.close()

        else:
            # es el dia de compra, el precio esta por encima de la sma
            if today.day_of_week == self.day_to_buy and self.data.Close[-1] > self.sma[-1]:
                sl_price = self.data.Close[-1] - self.data.Close[-1] *  (self.percentage_price_sl / 100)
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.buy(
                    size=units,
                    sl=sl_price,
                )

In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    

In [9]:
stats_per_symbol = {}

strategy = DayPerWeek

wfo_performance = pd.DataFrame()
for ticker, intervals in symbols.items():
    for interval in intervals:
        print(ticker, interval)
        
        lookback_bars = 1000
        validation_bars = 250
        warmup_bars = 200

        params = {
            'day_to_buy': list(np.arange(0, 5, 1)),
            'percentage_price_sl': list(np.arange(2, 20, 2)),
            'maximize': optim_func_2
        }

        try:
            wfo_stats = walk_forward(
                strategy,
                symbols[ticker][interval], 
                lookback_bars=lookback_bars,
                validation_bars=validation_bars,
                warmup_bars=warmup_bars, 
                params=params,
                commission=7e-4, 
                margin=1/30, 
                cash=100_000,
                verbose=False
            )
            
            stats_per_symbol[ticker] = {}
            stats_per_symbol[ticker][interval] = wfo_stats
            
            df_equity = wfo_stats['_equity']
            
            plot_full_equity_curve(df_equity, title=f'{ticker}, {interval}')        
            
            # Calculo el stability ratio
            x = np.arange(df_equity.shape[0]).reshape(-1, 1)
            reg = LinearRegression().fit(x, df_equity.Equity)
            stability_ratio = reg.score(x, df_equity.Equity)
            

            # Extraigo metricas
            df_stats = pd.DataFrame({
                'strategy':[strategy.__name__],
                'ticker':[ticker],
                'interval':[interval],
                'stability_ratio':[stability_ratio],
                'return':[wfo_stats['Return [%]']],
                'final_eq':[wfo_stats['Equity Final [$]']],
                'drawdown':[wfo_stats['Max. Drawdown [%]']],
                'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
                'win_rate':[wfo_stats['Win Rate [%]']], 
                'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
                'trades':[wfo_stats['# Trades']],
                'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
                'exposure':[wfo_stats['Exposure Time [%]']],
                'final_equity':[wfo_stats['Equity Final [$]']],
                'Duration':[wfo_stats['Duration']],

            })

            wfo_performance = pd.concat([wfo_performance, df_stats])
        
        except:
            print(f'No se pudo ejecutar para el ticker {ticker}')

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)

AAPLm 16388
{'day_to_buy': 0, 'percentage_price_sl': 6}
{'day_to_buy': 0, 'percentage_price_sl': 6}
{'day_to_buy': 0, 'percentage_price_sl': 4}
{'day_to_buy': 0, 'percentage_price_sl': 4}


ABTm 16388
{'day_to_buy': 2, 'percentage_price_sl': 2}


AMZNm 16388
{'day_to_buy': 0, 'percentage_price_sl': 8}
{'day_to_buy': 2, 'percentage_price_sl': 2}
{'day_to_buy': 2, 'percentage_price_sl': 6}
{'day_to_buy': 2, 'percentage_price_sl': 6}


BACm 16388
{'day_to_buy': 3, 'percentage_price_sl': 4}
{'day_to_buy': 0, 'percentage_price_sl': 6}
{'day_to_buy': 3, 'percentage_price_sl': 4}
{'day_to_buy': 4, 'percentage_price_sl': 2}


BMYm 16388
{'day_to_buy': 1, 'percentage_price_sl': 2}


...

XPEVm 16388
{'day_to_buy': 4, 'percentage_price_sl': 2}
{'day_to_buy': 4, 'percentage_price_sl': 2}
{'day_to_buy': 4, 'percentage_price_sl': 2}
{'day_to_buy': 4, 'percentage_price_sl': 2}


YUMCm 16388
{'day_to_buy': 1, 'percentage_price_sl': 4}


ZTOm 16388
{'day_to_buy': 2, 'percentage_price_sl': 4}


METAm 16388
{'day_to_buy': 4, 'percentage_price_sl': 4}
{'day_to_buy': 4, 'percentage_price_sl': 2}
{'day_to_buy': 2, 'percentage_price_sl': 2}
{'day_to_buy': 4, 'percentage_price_sl': 18}
{'day_to_buy': 1, 'percentage_price_sl': 4}
{'day_to_buy': 1, 'percentage_price_sl': 2}
{'day_to_buy': 1, 'percentage_price_sl': 2}
{'day_to_buy': 0, 'percentage_price_sl': 2}
{'day_to_buy': 1, 'percentage_price_sl': 6}


,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,DayPerWeek,TMEm,16388,0.959991,2.650268,102650.267672,0.451790,45,89.473684,3.175098,19,2.181643,9.666667,102650.267672,945 days 04:00:00,5.866151,5.468761
0,DayPerWeek,WMTm,16388,0.790537,1.563790,101563.789543,0.426012,71,55.555556,2.020888,18,0.677166,11.466667,101563.789543,945 days 04:00:00,3.670764,3.228923
0,DayPerWeek,ISRGm,16388,0.664233,8.727367,108727.367214,2.746307,69,68.750000,1.838145,16,1.032980,9.066667,108727.367214,945 days 04:00:00,3.177856,6.600232
0,DayPerWeek,LMTm,16388,0.838049,2.154017,102154.017497,0.838940,35,92.307692,1.983520,13,0.653441,7.466667,102154.017497,945 days 04:00:00,2.567546,3.091224
0,DayPerWeek,XPEVm,16388,0.583728,17.295093,117295.092760,7.341579,311,40.476190,1.011237,42,0.715780,4.311111,117295.092760,996 days 04:00:00,2.355773,7.798320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DayPerWeek,MAm,16388,0.900477,-6.982112,93017.887692,7.238418,196,42.857143,0.000000,14,-1.036558,5.533333,93017.887692,945 days 04:00:00,-0.964591,-2.295090
0,DayPerWeek,BEKEm,16388,0.487816,-2.501216,97498.783649,2.592650,204,53.846154,0.000000,13,-2.788840,7.600000,97498.783649,941 days 04:00:00,-0.964734,-1.837322
0,DayPerWeek,PEPm,16388,0.925016,-6.892981,93107.018777,6.901314,204,43.750000,0.000000,16,-0.895504,6.266667,93107.018777,981 days 20:00:00,-0.998793,-2.471650
0,DayPerWeek,CMCSAm,16388,0.108089,-1.575991,98424.008722,1.575991,42,0.000000,0.000000,2,-3.201269,0.333333,98424.008722,945 days 04:00:00,-1.000000,-0.672131


In [14]:
filtered_wfo_performance = wfo_performance[
    (wfo_performance.stability_ratio > 0.7)
    & (wfo_performance['return/dd'] > 1)
].sort_values(by='return/dd', ascending=False)
filtered_wfo_performance

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,DayPerWeek,TMEm,16388,0.959991,2.650268,102650.267672,0.451790,45,89.473684,3.175098,19,2.181643,9.666667,102650.267672,945 days 04:00:00,5.866151,5.468761
0,DayPerWeek,WMTm,16388,0.790537,1.563790,101563.789543,0.426012,71,55.555556,2.020888,18,0.677166,11.466667,101563.789543,945 days 04:00:00,3.670764,3.228923
0,DayPerWeek,LMTm,16388,0.838049,2.154017,102154.017497,0.838940,35,92.307692,1.983520,13,0.653441,7.466667,102154.017497,945 days 04:00:00,2.567546,3.091224
0,DayPerWeek,PYPLm,16388,0.824721,2.498125,102498.125290,1.887171,125,66.666667,0.638027,27,0.415802,5.644444,102498.125290,983 days 08:00:00,1.323741,2.883191
0,DayPerWeek,Cm,16388,0.855260,5.342852,105342.852271,4.128783,175,68.000000,0.979675,50,0.509422,10.577778,105342.852271,997 days 04:00:00,1.294050,4.095935
0,DayPerWeek,PDDm,16388,0.764479,5.954218,105954.217885,4.791627,116,61.538462,0.838371,52,0.691422,9.466667,105954.217885,994 days 04:00:00,1.242630,4.081751


# Montecarlo

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import montecarlo_statistics_simulation


# Crear una lista para almacenar los resultados de cada ticker
data_drawdown = []
data_return = []

dd_df = pd.DataFrame()
ret_df = pd.DataFrame()

for ticker in filtered_wfo_performance.ticker.tolist():
    print(f"Procesando ticker: {ticker}")
    trades_history = stats_per_symbol[ticker][16388]._trades
    
    # Simulación de Montecarlo para cada ticker (datos agregados)
    mc, dd, ret = montecarlo_statistics_simulation(trade_history=trades_history, n_simulations=100_000, initial_equity=100_000, threshold_ruin=0.8, return_raw_curves=True)
    
    display(mc)
    
    dd['ticker'] = ticker
    ret['ticker'] = ticker
    
    dd_df = pd.concat([dd_df, dd])
    ret_df = pd.concat([ret_df, ret])



ModuleNotFoundError: No module named 'utils'

In [ ]:
# Configurar el gráfico con matplotlib y seaborn
plt.figure(figsize=(25, 8))
sns.boxplot(data=dd_df, x="ticker", y="Drawdown (%)")
plt.title("Comparación de Drawdown (%) entre Tickers")

y_max = dd_df["Drawdown (%)"].max()  # Valor máximo en el eje Y
y_min = dd_df["Drawdown (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 2  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Generar el gráfico
plt.figure(figsize=(25, 8))
sns.boxplot(data=ret_df, x="ticker", y="Final Return (%)")
plt.title("Comparación de Retorno (%) entre Tickers")

# Configurar ticks mayores con más números
y_max = ret_df["Final Return (%)"].max()  # Valor máximo en el eje Y
y_min = ret_df["Final Return (%)"].min()  # Valor mínimo en el eje Y
tick_interval = 10  # Intervalo deseado entre números en el eje Y

# Configurar los ticks mayores en el eje Y
plt.yticks(np.arange(y_min, y_max + tick_interval, tick_interval))

# Activar la cuadrícula
plt.grid(True, linestyle='--', which='both', color='grey', alpha=0.7)

# Mostrar el gráfico
plt.show()


In [ ]:
    def max_drawdown(equity_curve):
        running_max = np.maximum.accumulate(equity_curve)
        drawdown = (equity_curve - running_max) / running_max
        return np.min(drawdown) * 100  # Convertir el drawdown a porcentaje
    
    max_drawdown([10])